## Imports / Globals

In [1]:
import pickle
import pandas as pd
from os import listdir
from os.path import isfile, join
from tqdm import tqdm
import numpy as np
from sklearn.model_selection import train_test_split

np.random.seed(42)

In [2]:
# path = '/Users/thomas/Downloads/nturgb+d_skeletons'
path = 'D:\\Datasets\\Motion Privacy\\NTU RGB+D 120\\Skeleton Data'

## Data organization

### Input format
needs to be updated

[J0 X, J0 Y, J0 Z, J1 X, J1 Y, J1 Z, ..., J25 Z]

### Output format
One Hot encoded action classification len(Y) = 120

In [3]:
def load_files():
    # Read the files
    files = [f for f in listdir(path) if isfile(join(path, f))]

    # Get stats for each file based on name
    files_ = []
    for file in files:
        data = {'file': file,
                's': file[0:4],
                'c': file[4:8],
                'p': file[8:12],
                'r': file[12:16],
                'a': file[16:20]
                }
        files_.append(data)

    return files_
files_ = None

In [11]:
# Attempt to load X and Y from pickle before generating them
X = {}
try:
    print('Attempting to load X from pickle')
    with open('data/X_action.pkl', 'rb') as f:
        X = pickle.load(f)
    print('X loaded from pickle')
except:
    print('Could not load X and Y, generating them now')

    # Load the files
    if files_ is None:
        files_ = load_files()

    # Generate X and Y
    for file_ in tqdm(files_, desc='Files Parsed', position=0):
        try:
            file = join(path, file_['file'])
            data = open(file, 'r')
            lines = data.readlines()
            frames_count = int(lines.pop(0).replace('\n', ''))
            file_['frames'] = frames_count
        except UnicodeDecodeError:  # .DS_Store file
            print('UnicodeDecodeError: ', file)
            continue

        # Get P and add to X if not already there
        a = file_['a']
        if a not in X:
            X[a] = []

        # Hold all frames for the file (action)
        frames = []

        # To validate the video is good
        good = True

        for f in tqdm(range(frames_count), desc='Frames Parsed', position=1, leave=False):
            try:
                # Get actor count
                actors = int(lines.pop(0).replace('\n', ''))

                # Hold frame info
                frame = []

                # Iterate through actors
                for ac in range(actors):
                    # Get actor info
                    t = lines.pop(0)

                    # Get joint count
                    joint_count = int(lines.pop(0).replace('\n', ''))

                    # Get joint info
                    d = []
                    for j in range(joint_count):
                        joint = lines.pop(0).replace('\n', '').split(' ')
                        d.extend(joint[0:3])

                    # Convert to float
                    d = [float(i) for i in d]

                    # Skip if not 25 joints
                    if len(d) != 75:
                        good = False
                        break

                    # Append to frame
                    frame.extend(d)

                # Convert to numpy array
                frame = np.array(frame, dtype=np.float32)

                # Pad frame to 150 (for 2 actors)
                if len(frame) < 150:
                    frame = np.pad(frame, (0, 150-len(frame)), 'constant')

                # Append to X and Y
                frames.append(frame)
            except:
                break

        if not good:
            continue

        if type(frames) != list:
            print('Not a list: ', type(frames), frames)
            continue

        # Convert to numpy array
        frames = np.array(frames, dtype=np.ndarray)

        # Validate frames size
        if len(frames.shape) != 2 or frames.shape[1] < 150:
            continue

        # Pad X size to 300 frames (300 is max frames in dataset)
        # Each frame is 25 joints * 3 coordinates * 2 potential actors = 150
        # For the real time attack model, we can make a new prediction every 300 frames (5 second @ 60fps) to align with this
        if frames.shape[0] < 300:
            frames = np.pad(
                frames, ((0, 300-frames.shape[0]), (0, 0)), 'constant')

        # Validate frames size
        if frames.shape != (300, 150):
            continue

        # Ensure the frame isnt all zeros
        if np.sum(frames) == 0:
            continue

        # Add frames to X
        X[a].append(frames)

    # Convert to numpy arrays
    for a in X:
        X[a] = np.array(X[a]).astype(np.float32)

    print('X Generated, saving to pickle...')

    # Save the data
    with open('data/X_action.pkl', 'wb') as f:
        pickle.dump(X, f)

    print('X Saved to pickle')

# Print Lengths
# for p in X:
#     print(p, len(X[p]))

Attempting to load X from pickle
Could not load X and Y, generating them now


Files Parsed: 100%|██████████| 114480/114480 [1:21:47<00:00, 23.33it/s]


X Generated, saving to pickle...
X Saved to pickle


In [12]:
def process_data():
    x=[]
    y=[]
    for action in X:
        # Get the action index
        action_index = int(action[1:4])-1
        # Convert to onehot encoding
        y_ = np.zeros(120)
        y_[action_index] = 1
        # Append to X and Y
        y.extend([y_]*len(X[action]))
        x.extend(X[action])
    return x,y

X_, Y_ = process_data()

In [13]:
# Split into train and test
train_x, test_x, train_y, test_y = train_test_split(X_, Y_, test_size=0.3, random_state=42)
# Split into validation and test
val_x, test_x, val_y, test_y = train_test_split(test_x, test_y, test_size=0.5, random_state=42)

## SGN

All code in this section is adapted from Microsoft's SGN. [Github](https://github.com/microsoft/SGN)

In [14]:
import time
import shutil
import os
import os.path as osp
import csv
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim.lr_scheduler import MultiStepLR
from model import SGN
from data import NTUDataLoaders, AverageMeter
from util import make_dir, get_num_classes

Exception ignored in: <function tqdm.__del__ at 0x0000014FB83FCEE0>
Traceback (most recent call last):
  File "c:\Users\Carrt\AppData\Local\Programs\Python\Python39\lib\site-packages\tqdm\std.py", line 1162, in __del__
    self.close()
  File "c:\Users\Carrt\AppData\Local\Programs\Python\Python39\lib\site-packages\tqdm\std.py", line 1291, in close
    if self.last_print_t < self.start_t + self.delay:
AttributeError: 'tqdm' object has no attribute 'last_print_t'


In [15]:
# Hyperparameters/Tuning Parameters
network='SGN'
dataset='NTU'
start_epoch=0
case=1 # 0 = Gender, 1 = Action
batch_size=64
max_epochs=120
monitor='val_acc'
lr=0.001
weight_decay=0.0001
lr_factor=0.1
workers=16
print_freq = 20
do_train=1
seg=20

In [24]:
def train(train_loader, model, criterion, optimizer, epoch):
    losses = AverageMeter()
    acces = AverageMeter()
    model.train()

    for i, (inputs, target) in enumerate(train_loader):

        output = model(inputs.cuda())
        target = target.cuda()
        loss = criterion(output, target)

        # measure accuracy and record loss
        acc = accuracy(output.data, target)
        losses.update(loss.item(), inputs.size(0))
        acces.update(acc[0], inputs.size(0))

        # backward
        optimizer.zero_grad()  # clear gradients out before each mini-batch
        loss.backward()
        optimizer.step()

        if (i + 1) % print_freq == 0:
            print('Epoch-{:<3d} {:3d} batches\t'
                  'loss {loss.val:.4f} ({loss.avg:.4f})\t'
                  'accu {acc.val:.3f} ({acc.avg:.3f})'.format(
                      epoch + 1, i + 1, loss=losses, acc=acces))

    return losses.avg, acces.avg


def validate(val_loader, model, criterion):
    losses = AverageMeter()
    acces = AverageMeter()
    model.eval()

    for i, (inputs, target) in enumerate(val_loader):
        with torch.no_grad():
            output = model(inputs.cuda())
        target = target.cuda()
        with torch.no_grad():
            loss = criterion(output, target)

        # measure accuracy and record loss
        acc = accuracy(output.data, target)
        losses.update(loss.item(), inputs.size(0))
        acces.update(acc[0], inputs.size(0))

    return losses.avg, acces.avg


def test(test_loader, model, checkpoint, lable_path, pred_path):
    acces = AverageMeter()
    # load learnt model that obtained best performance on validation set
    model.load_state_dict(torch.load(checkpoint)['state_dict'])
    model.eval()

    label_output = list()
    pred_output = list()

    t_start = time.time()
    for i, t in enumerate(test_loader):
        inputs = t[0]
        target = t[1]
        with torch.no_grad():
            output = model(inputs.cuda())
            output = output.view(
                (-1, inputs.size(0)//target.size(0), output.size(1)))
            output = output.mean(1)

        label_output.append(target.cpu().numpy())
        pred_output.append(output.cpu().numpy())

        acc = accuracy(output.data, target.cuda())
        acces.update(acc[0], inputs.size(0))

    label_output = np.concatenate(label_output, axis=0)
    np.savetxt(lable_path, label_output, fmt='%d')
    pred_output = np.concatenate(pred_output, axis=0)
    np.savetxt(pred_path, pred_output, fmt='%f')

    print('Test: accuracy {:.3f}, time: {:.2f}s'
          .format(acces.avg, time.time() - t_start))


def accuracy(output, target):
    batch_size = target.size(0)
    _, pred = output.topk(1, 1, True, True)
    pred = pred.t()
    target = torch.argmax(target, dim=1)  # Add this line to convert one-hot targets to class indices
    correct = pred.eq(target.view(1, -1).expand_as(pred))
    correct = correct.view(-1).float().sum(0, keepdim=True)
    return correct.mul_(100.0 / batch_size)



def save_checkpoint(state, filename='checkpoint.pth.tar', is_best=False):
    torch.save(state, filename)
    if is_best:
        shutil.copyfile(filename, 'model_best.pth.tar')


def get_n_params(model):
    pp = 0
    for p in list(model.parameters()):
        nn = 1
        for s in list(p.size()):
            nn = nn*s
        pp += nn
    return pp


class LabelSmoothingLoss(nn.Module):
    def __init__(self, classes, smoothing=0.0, dim=-1):
        super(LabelSmoothingLoss, self).__init__()
        self.confidence = 1.0 - smoothing
        self.smoothing = smoothing
        self.cls = classes
        self.dim = dim

    def forward(self, pred, target):
        pred = pred.log_softmax(dim=self.dim)
        with torch.no_grad():
            target = torch.argmax(target, dim=1)  # Add this line to convert one-hot targets to class indices
            true_dist = torch.zeros_like(pred)
            true_dist.fill_(self.smoothing / (self.cls - 1))
            true_dist.scatter_(1, target.data.unsqueeze(1), self.confidence)
        return torch.mean(torch.sum(-true_dist * pred, dim=self.dim))



In [ ]:
def main():
    num_classes = get_num_classes(dataset, 1)
    model = SGN(num_classes, dataset, seg, batch_size, do_train)

    total = get_n_params(model)
    # print(model)
    print('The number of parameters: ', total)
    print('The modes is:', network)

    if torch.cuda.is_available():
        print('It is using GPU!')
        model = model.cuda()

    criterion = LabelSmoothingLoss(num_classes, smoothing=0.1).cuda()
    optimizer = optim.Adam(model.parameters(), lr=lr,
                           weight_decay=weight_decay)

    if monitor == 'val_acc':
        mode = 'max'
        monitor_op = np.greater
        best = -np.Inf
        str_op = 'improve'
    elif monitor == 'val_loss':
        mode = 'min'
        monitor_op = np.less
        best = np.Inf
        str_op = 'reduce'

    scheduler = MultiStepLR(optimizer, milestones=[60, 90, 110], gamma=0.1)
    # Data loading
    ntu_loaders = NTUDataLoaders(dataset, case, seg=seg, train_X=train_x, train_Y=train_y, test_X=test_x, test_Y=test_y, val_X=val_x, val_Y=val_y, aug=0)
    train_loader = ntu_loaders.get_train_loader(batch_size, workers)
    val_loader = ntu_loaders.get_val_loader(batch_size, workers)
    train_size = ntu_loaders.get_train_size()
    val_size = ntu_loaders.get_val_size()

    test_loader = ntu_loaders.get_test_loader(32, workers)

    print('Train on %d samples, validate on %d samples' %
          (train_size, val_size))

    best_epoch = 0
    output_dir = make_dir(dataset)

    save_path = os.path.join(output_dir, network)
    if not os.path.exists(save_path):
        os.makedirs(save_path)

    checkpoint = osp.join(save_path, '%s_best.pth' % case)
    earlystop_cnt = 0
    csv_file = osp.join(save_path, '%s_log.csv' % case)
    log_res = list()

    lable_path = osp.join(save_path, '%s_lable.txt' % case)
    pred_path = osp.join(save_path, '%s_pred.txt' % case)

    # Training
    if do_train == 1:
        for epoch in range(start_epoch, max_epochs):

            print(epoch, optimizer.param_groups[0]['lr'])

            t_start = time.time()
            train_loss, train_acc = train(
                train_loader, model, criterion, optimizer, epoch)
            val_loss, val_acc = validate(val_loader, model, criterion)
            log_res += [[train_loss, train_acc.cpu().numpy(),
                         val_loss, val_acc.cpu().numpy()]]

            print('Epoch-{:<3d} {:.1f}s\t'
                  'Train: loss {:.4f}\taccu {:.4f}\tValid: loss {:.4f}\taccu {:.4f}'
                  .format(epoch + 1, time.time() - t_start, train_loss, train_acc, val_loss, val_acc))

            current = val_loss if mode == 'min' else val_acc

            # store tensor in cpu
            current = current.cpu()

            if monitor_op(current, best):
                print('Epoch %d: %s %sd from %.4f to %.4f, '
                      'saving model to %s'
                      % (epoch + 1, monitor, str_op, best, current, checkpoint))
                best = current
                best_epoch = epoch + 1
                save_checkpoint({
                    'epoch': epoch + 1,
                    'state_dict': model.state_dict(),
                    'best': best,
                    'monitor': monitor,
                    'optimizer': optimizer.state_dict(),
                }, checkpoint)
                earlystop_cnt = 0
            else:
                print('Epoch %d: %s did not %s' % (epoch + 1, monitor, str_op))
                earlystop_cnt += 1

            scheduler.step()

        print('Best %s: %.4f from epoch-%d' % (monitor, best, best_epoch))
        with open(csv_file, 'w') as fw:
            cw = csv.writer(fw)
            cw.writerow(['loss', 'acc', 'val_loss', 'val_acc'])
            cw.writerows(log_res)
        print('Save train and validation log into into %s' % csv_file)

    # Test
    model = SGN(num_classes, dataset, seg, batch_size, 0)
    model = model.cuda()
    test(test_loader, model, checkpoint, lable_path, pred_path)

main()